# MemoryVectorStore

LangChain 提供了一个内存中的临时向量存储，它将嵌入存储在内存中，并通过精确的线性搜索查找最相似的嵌入。默认的相似性度量是余弦相似度，但可以更改为 [ml-distance](https://mljs.github.io/distance/modules/similarity.html) 支持的任何相似性度量。

由于其旨在用于演示，因此尚不支持 ID 或删除功能。

本指南提供了开始使用内存中的 [`向量存储`](/docs/concepts/#vectorstores) 的快速概述。如需详细了解所有 `MemoryVectorStore` 功能和配置，请参阅 [API 参考文档](https://api.js.langchain.com/classes/langchain.vectorstores_memory.MemoryVectorStore.html)。

## 概览

### 集成详情

| 类 | 包 | Python支持 |  包最新版本 |
| :--- | :--- | :---: | :---: |
| [`MemoryVectorStore`](https://api.js.langchain.com/classes/langchain.vectorstores_memory.MemoryVectorStore.html) | [`langchain`](https://www.npmjs.com/package/langchain) | ❌ |  ![NPM - 版本](https://img.shields.io/npm/v/langchain?style=flat-square&label=%20&) |

## 环境配置

如需使用内存中的向量存储，你需要安装 `langchain` 包：

本指南还将使用 [OpenAI 嵌入模型](/docs/integrations/text_embedding/openai)，这要求你安装 `@langchain/openai` 集成包。当然，你也可以选择使用[其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain @langchain/openai @langchain/core
</Npm2Yarn>
```

### 凭证

使用内存中的向量存储不需要任何凭证。

如果你在本指南中使用 OpenAI 嵌入模型，则还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动化追踪，也可以取消以下 [LangSmith](https://docs.smith.langchain.com/) API 密钥设置的注释：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

In [1]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const vectorStore = new MemoryVectorStore(embeddings);

## 管理向量存储

### 向向量存储中添加条目

In [2]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const documents = [document1, document2, document3];

await vectorStore.addDocuments(documents);

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能会希望在链或代理运行期间查询它。

### 直接查询

执行一个简单的相似性搜索可以按如下方式进行：

In [3]:
const filter = (doc) => doc.metadata.source === "https://example.com";

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter)

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


过滤器是可选的，必须是一个谓词函数，以文档作为输入，并根据该文档是否应当被返回，输出 `true` 或 `false`。
如果你想执行相似性搜索并获得相应的分数，可以运行：

In [4]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.165] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.148] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

您还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在您的链中更方便地使用：

In [5]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});

await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 最大边缘相关性

该向量存储还支持最大边缘相关性（MMR）技术，该技术首先通过经典的相似性搜索获取更多的结果（由 `searchKwargs.fetchK` 指定），然后重新排序以提高多样性，并返回排名前 `k` 的结果。这有助于防止重复信息：

In [6]:
const mmrRetriever = vectorStore.asRetriever({
  searchType: "mmr",
  searchKwargs: {
    fetchK: 10,
  },
  // Optional filter
  filter: filter,
  k: 2,
});

await mmrRetriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Buildings are made out of brick',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何：使用 RAG 进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考

如需详细了解所有 `MemoryVectorStore` 的功能和配置，请访问 [API 参考](https://api.js.langchain.com/classes/langchain.vectorstores_memory.MemoryVectorStore.html)。